In [34]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

os.environ["OPENAI_API_KEY"] = "sk-proj-Woh8SFgZ0Z2kVew3diIYT3BlbkFJSpojO8dFcZCX1Lu24XdP"

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

In [52]:
import pandas as pd

# Define the path to your Excel file
excel_file_path = 'data/CanTho_Tourist_Attractions.xlsx'

# Load the Excel file
excel_data = pd.ExcelFile(excel_file_path)

# Iterate through each sheet in the Excel file
for sheet_name in excel_data.sheet_names:
    # Read the sheet
    data = pd.read_excel(excel_data, sheet_name=sheet_name)
    
    # Define the path to save the CSV file
    csv_file_path = 'data/CanTho_Tourist_Attractions.csv'
    
    # Save the data to a CSV file, without the index column
    data.to_csv(csv_file_path, index=False)

    # Print the file path to confirm it's saved
    print(f'Saved: {csv_file_path}')


Saved: data/CanTho_Tourist_Attractions.csv


In [53]:
recipe_df = pd.read_csv("data/CanTho_Tourist_Attractions.csv")

recipe_df.head()

,Question,Answer,Source
0,"About Can Tho, Vietnam",Nestled in the verdant embrace of the Mekong D...,https://vinpearl.com/en/can-tho-vietnam-the-ul...
1,How is Can Tho's weather? The best time to vis...,Can Tho is typical of a tropical climate with ...,https://vinpearl.com/en/can-tho-vietnam-the-ul...
2,"Must-visit tourist attractions in Can Tho, Vie...",Cai Rang Floating Market\nLocation: downstream...,https://vinpearl.com/en/can-tho-vietnam-the-ul...
3,"How to get to Can Tho City, Vietnam?\n",As an important industrial center of the Mekon...,https://vinpearl.com/en/can-tho-vietnam-the-ul...
4,Where to stay in Can Tho City?,"Depending on preferences and budgets, visitors...",https://vinpearl.com/en/can-tho-vietnam-the-ul...


In [54]:
training_data = []

system_message = "You are an expert in providing travel assistance for Can Tho. Your role is to offer detailed information, tips, and guidance to travelers interested in exploring Can Tho, Vietnam. You provide insights into the best places to visit, local cuisine, cultural highlights, and practical travel advice to ensure visitors have a memorable and smooth experience in Can Tho."

def create_user_message(row):
    return f"""Question: {row['Question']}"""

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["Answer"]})

    return {"messages": messages}

print(prepare_example_conversation(recipe_df.iloc[0]))

{'messages': [{'role': 'system', 'content': 'You are an expert in providing travel assistance for Can Tho. Your role is to offer detailed information, tips, and guidance to travelers interested in exploring Can Tho, Vietnam. You provide insights into the best places to visit, local cuisine, cultural highlights, and practical travel advice to ensure visitors have a memorable and smooth experience in Can Tho.'}, {'role': 'user', 'content': 'Question: About Can Tho, Vietnam'}, {'role': 'assistant', 'content': "Nestled in the verdant embrace of the Mekong Delta, Can Tho beckons travelers with a symphony of sights, sounds, and flavors that epitomize the essence of Vietnam. This enchanting city is a gateway to the region's rich cultural heritage and timeless traditions. The iconic floating market stands as a testament to the city's vibrant trading history, where boats laden with fresh fruits, vegetables, and handicrafts create a lively tapestry on the water. Beyond the market's bustling ener

In [64]:
# use the first 100 rows of the dataset for training
training_df = recipe_df.loc[0:10]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are an expert in providing travel assistance for Can Tho. Your role is to offer detailed information, tips, and guidance to travelers interested in exploring Can Tho, Vietnam. You provide insights into the best places to visit, local cuisine, cultural highlights, and practical travel advice to ensure visitors have a memorable and smooth experience in Can Tho.'}, {'role': 'user', 'content': 'Question: About Can Tho, Vietnam'}, {'role': 'assistant', 'content': "Nestled in the verdant embrace of the Mekong Delta, Can Tho beckons travelers with a symphony of sights, sounds, and flavors that epitomize the essence of Vietnam. This enchanting city is a gateway to the region's rich cultural heritage and timeless traditions. The iconic floating market stands as a testament to the city's vibrant trading history, where boats laden with fresh fruits, vegetables, and handicrafts create a lively tapestry on the water. Beyond the market's bustling ener

In [65]:
validation_df = recipe_df.loc[11:]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

In [66]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [67]:
training_file_name = "cantho_tourist_attraction_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "cantho_tourist_attraction_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [ ]:
# print the first 5 lines of the training file
!head -n 5 cantho_tourist_attraction_finetune_training.jsonl

In [69]:
with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-T5WeuRkkCJs46Z5qswYUCilF
Validation file ID: file-k5BRWBjLFVIs1JEAPDpVRMo3


In [70]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="cantho_tourist",
    # hyperparameters={
    #     "learning_rate": 1e-4,
    #     "batch_size": 4,
    #     "num_epochs": 3
    # }
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-vDHXWjsLvEra6IQRra9XqpLL
Status: validating_files


In [71]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-vDHXWjsLvEra6IQRra9XqpLL
Status: validating_files
Trained Tokens: None


In [72]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-vDHXWjsLvEra6IQRra9XqpLL
Validating training file: file-T5WeuRkkCJs46Z5qswYUCilF and validation file: file-k5BRWBjLFVIs1JEAPDpVRMo3


In [73]:
test_df = recipe_df.loc[6:]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

print(test_messages)
print(test_row["Answer"])

[{'role': 'system', 'content': 'You are an expert in providing travel assistance for Can Tho. Your role is to offer detailed information, tips, and guidance to travelers interested in exploring Can Tho, Vietnam. You provide insights into the best places to visit, local cuisine, cultural highlights, and practical travel advice to ensure visitors have a memorable and smooth experience in Can Tho.'}, {'role': 'user', 'content': 'Question: new landmark for Can Tho City, the Mekong Delta Tower.'}]
magine a new landmark for Can Tho City, the Mekong Delta Tower. Designed to reflect the flowing waters and vibrant culture of the Mekong Delta, this modern marvel would stand tall at 350 meters, making it a standout feature in the cityscape. Its structure would mimic the sinuous curves of a river, crafted from glass and steel that shimmer like the surface of the water under the sun.

The Mekong Delta Tower would feature multiple levels accessible by both stairs and panoramic elevators. The lower l

In [74]:
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:recipe-ner:9Lnbj8iK", messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': 'you must provide a model parameter', 'type': 'invalid_request_error', 'param': None, 'code': None}}